# Iteration 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import math 
from pandas.core import shared_docs
import scipy.stats

# visualization
import seaborn as sns

# similarity
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, kendalltau

In [ ]:
def alpha_score(userARec, userBRec, userCRec, GroupRec):
  #Calculating the satisfaction score for every user
  satUserA = np.sum(GroupRec)/np.sum(userARec)
  satUserB = np.sum(GroupRec)/np.sum(userBRec)
  satUserC = np.sum(GroupRec)/np.sum(userCRec)
  satUsersList = [satUserA, satUserB, satUserC]

  #Calculating the Group Satisfaction Score
  group_sat = np.mean(satUsersList)

  #Kendall Tau between the users so as to find out the disagreement among the users  
  ktAB, p_valAB = kendalltau(userARec, userBRec)
  ktBC, p_valBC = kendalltau(userBRec, userCRec)
  ktAC, p_valAC = kendalltau(userARec, userCRec)

  #Kendall Tau between users and group as done in assignment 2
  tauAAVG_userA, p_valueA_userA = kendalltau(userARec, GroupRec)
  tauAAVG_userB, p_valueA_userB = kendalltau(userBRec, GroupRec)
  tauAAVG_userC, p_valueA_userC = kendalltau(userCRec, GroupRec)

  #Calculating the means for two scenarios ;- among users, between user individual list and generated movie list
  ktOnlyUsers = np.mean([ktAB, ktBC, ktAC])
  ktOverall = np.mean([tauAAVG_userA, tauAAVG_userB, tauAAVG_userC])

  #Group disagreement
  #Difference of the result of two scenarios
  alpha = abs(ktOnlyUsers - ktOverall)

  return satUserA, satUserB, satUserC, alpha, group_sat

In [ ]:
import numpy as np
rating = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/movies.csv")
df_assignment_3 = pd.merge(rating, movies, on='movieId', how='inner')
chunks = np.array_split(df_assignment_3, 2)
df_big_chunk = chunks[0]

In [ ]:
matrix_item_assignment_3 = df_big_chunk.pivot_table(index='title', columns='userId', values='rating')
matrix_item_norm_assignment_3 = matrix_item_assignment_3.subtract(matrix_item_assignment_3.mean(axis=1), axis = 0)
df_item_similarity_cosine_asign_3 = pd.DataFrame(cosine_similarity(matrix_item_norm_assignment_3.fillna(0.0000001)))
item_similarity_pearson_assignment_3 = matrix_item_norm_assignment_3.T.corr()
df_item_similarity_cosine_asign_3.columns = item_similarity_pearson_assignment_3.columns
df_item_similarity_cosine_asign_3.index = item_similarity_pearson_assignment_3.index

In [ ]:
# Item-based recommendation function
def item_based_rec_assignment_3(picked_userid, number_of_similar_items, number_of_recommendations):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_item_norm_assignment_3[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title'].values.tolist()

  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_item_norm_assignment_3[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  

  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    # picked_movie_similarity_score = item_similarity_pearson[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    picked_movie_similarity_score = df_item_similarity_cosine_asign_3[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})

    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    # predicted_rating = round(np.average(picked_userid_watched_similarity['rating']),6)
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

In [ ]:
#recommendations for user 110

print("Recommended movies for user 110")
recommended_movies_user_110 = item_based_rec_assignment_3(picked_userid=110, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_110_df = pd.DataFrame(recommended_movies_user_110)
recommended_movies_user_110_df_list = recommended_movies_user_110_df[1].values.tolist()

Recommended movies for user 110


In [ ]:
print("Recommended movies for user 250")
recommended_movies_user_250 = item_based_rec_assignment_3(picked_userid=250, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_250_df = pd.DataFrame(recommended_movies_user_250)
recommended_movies_user_250_df_list = recommended_movies_user_250_df[1].values.tolist()

Recommended movies for user 250


In [ ]:
print("Recommended movies for user 401")
recommended_movies_user_401 = item_based_rec_assignment_3(picked_userid=401, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_401_df = pd.DataFrame(recommended_movies_user_401)
recommended_movies_user_401_df_list = recommended_movies_user_401_df[1].values.tolist()

Recommended movies for user 401


In [ ]:
output_list_group_3 = []
user_id_lists = [110,250,401]
for user in user_id_lists:
    recommended_movie = item_based_rec_assignment_3(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3.append(recommended_movie)

In [ ]:
#converting into dataframe
group_movies_df =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df = group_movies_df.rename(columns={0: 'movie', 1: 'score'})

In [ ]:
#using the average aggregation
df_mean = group_movies_df.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)

In [ ]:
recommendations_for_3_list = recommendations_for_3['score'].values.tolist()
recommendations_for_3_list_top20 = recommendations_for_3_list[:20]

In [ ]:
#get satisfaction score of first iteration
satA1, satB1, satC1, alpha1, group_sat1= alpha_score(recommended_movies_user_110_df_list, recommended_movies_user_250_df_list, recommended_movies_user_401_df_list, recommendations_for_3_list_top20)

In [ ]:
satA1, satB1, satC1, alpha1, group_sat1

(3.0918695664131377,
 1.0273396900142306,
 2.6968652229793735,
 0.0640479459038612,
 2.272024826468914)

# Iteration 2

In [ ]:
mini_chunks = np.array_split(chunks[1], 4)

In [ ]:

print(sorted(chunks[0]["userId"].unique()))
print(sorted(mini_chunks[0]["userId"].unique()))
print(sorted(mini_chunks[1]["userId"].unique()))
print(sorted(mini_chunks[2]["userId"].unique()))
print(sorted(mini_chunks[3]["userId"].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [ ]:
df_big_chunk_and_chunk_2_1 = pd.concat([chunks[0],mini_chunks[0]], ignore_index=True)

In [ ]:
matrix_item_assignment_3_iter2 = df_big_chunk_and_chunk_2_1.pivot_table(index='title', columns='userId', values='rating')
matrix_item_norm_assignment_3_iter2 = matrix_item_assignment_3_iter2.subtract(matrix_item_assignment_3_iter2.mean(axis=1), axis = 0)
df_item_similarity_cosine_asign_3_iter2 = pd.DataFrame(cosine_similarity(matrix_item_norm_assignment_3_iter2.fillna(0.0000001)))
item_similarity_pearson_assignment_3_iter2 = matrix_item_norm_assignment_3_iter2.T.corr()
df_item_similarity_cosine_asign_3_iter2.columns = item_similarity_pearson_assignment_3_iter2.columns
df_item_similarity_cosine_asign_3_iter2.index = item_similarity_pearson_assignment_3_iter2.index

In [ ]:
# Item-based recommendation function
def item_based_rec_assignment_3_iter2(picked_userid, number_of_similar_items, number_of_recommendations):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_item_norm_assignment_3_iter2[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title'].values.tolist()

  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_item_norm_assignment_3_iter2[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  

  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    # picked_movie_similarity_score = item_similarity_pearson[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    picked_movie_similarity_score = df_item_similarity_cosine_asign_3_iter2[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})

    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    # predicted_rating = round(np.average(picked_userid_watched_similarity['rating']),6)
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

In [ ]:
#recommendations for user 110

print("Recommended movies for user 110")
recommended_movies_user_110 = item_based_rec_assignment_3(picked_userid=110, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_110_df = pd.DataFrame(recommended_movies_user_110)
recommended_movies_user_110_df_list = recommended_movies_user_110_df[1].values.tolist()

Recommended movies for user 110


In [ ]:
print("Recommended movies for user 250")
recommended_movies_user_250 = item_based_rec_assignment_3(picked_userid=250, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_250_df = pd.DataFrame(recommended_movies_user_250)
recommended_movies_user_250_df_list = recommended_movies_user_250_df[1].values.tolist()

Recommended movies for user 250


In [ ]:
print("Recommended movies for user 401")
recommended_movies_user_401 = item_based_rec_assignment_3(picked_userid=401, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_401_df = pd.DataFrame(recommended_movies_user_401)
recommended_movies_user_401_df_list = recommended_movies_user_401_df[1].values.tolist()

Recommended movies for user 401


In [ ]:
output_list_group_3_iter2 = []
user_id_lists = [110,250,401]
for user in user_id_lists:
    recommended_movie = item_based_rec_assignment_3_iter2(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3_iter2.append(recommended_movie)

In [ ]:
#converting into dataframe
group_movies_df_iter2 =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df_iter2 = group_movies_df.rename(columns={0: 'movie', 1: 'score'})

In [ ]:
group_movies_df_iter2

,movie,score
0,Bridget Jones's Diary (2001),0.533963
1,Priceless (Hors de prix) (2006),0.522125
2,Speechless (1994),0.507170
3,Mallrats (1995),0.486892
4,Roommates (1995),0.457311
5,Princess Mononoke (Mononoke-hime) (1997),0.447869
6,"General, The (1926)",0.434774
7,Breakfast at Tiffany's (1961),0.422058
8,Lord of Illusions (1995),0.410941
9,Valentine's Day (2010),0.401128


Because alpha is really low from iteration 1 (0.06) for this iteration we will keep using the average mean method

In [ ]:
#using the average aggregation
df_mean = group_movies_df_iter2.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)
recommendations_for_3["score"] = recommendations_for_3["score"] * (1 - 0.0640479459038612)

In [ ]:
recommendations_for_3

,score
movie,
"Simple Twist of Fate, A (1994)",1.403384
Before and After (1996),1.402841
Safe (1995),1.344475
Mr. Mom (1983),1.209800
"Program, The (1993)",1.202854
L.I.E. (2001),1.196523
Sling Blade (1996),1.184112
Tales from the Hood (1995),1.183553
Match Point (2005),1.162936


In [ ]:
recommendations_for_3_list = recommendations_for_3['score'].values.tolist()
recommendations_for_3_list_top20 = recommendations_for_3_list[:20]

In [ ]:
#get satisfaction score of first iteration
satA1, satB1, satC1, alpha1, group_sat1= alpha_score(recommended_movies_user_110_df_list, recommended_movies_user_250_df_list, recommended_movies_user_401_df_list, recommendations_for_3_list_top20)

In [ ]:
satA1, satB1, satC1, alpha1, group_sat1

(2.8938416716817144,
 0.9615406931233097,
 2.5241365450679862,
 0.0640479459038612,
 2.1265063032910034)

As you can see, in iteration 2, the reason our alpha and satisfaction are the ssame becaue even if we add new chunk of data our functions still return the same list of recommended movies :( 

# Iteration 3

In [ ]:
print(sorted(chunks[0]["userId"].unique()))
print(sorted(mini_chunks[0]["userId"].unique()))
print(sorted(mini_chunks[1]["userId"].unique()))
print(sorted(mini_chunks[2]["userId"].unique()))
print(sorted(mini_chunks[3]["userId"].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [ ]:
df_big_chunk_and_chunk_2_1 = pd.concat([chunks[0],mini_chunks[0],mini_chunks[1]], ignore_index=True)

In [ ]:
matrix_item_assignment_3_iter3 = df_big_chunk_and_chunk_2_1.pivot_table(index='title', columns='userId', values='rating')
matrix_item_norm_assignment_3_iter3 = matrix_item_assignment_3_iter3.subtract(matrix_item_assignment_3_iter3.mean(axis=1), axis = 0)
df_item_similarity_cosine_asign_3_iter3 = pd.DataFrame(cosine_similarity(matrix_item_norm_assignment_3_iter3.fillna(0.0000001)))
item_similarity_pearson_assignment_3_iter3 = matrix_item_norm_assignment_3_iter3.T.corr()
df_item_similarity_cosine_asign_3_iter3.columns = item_similarity_pearson_assignment_3_iter3.columns
df_item_similarity_cosine_asign_3_iter3.index = item_similarity_pearson_assignment_3_iter3.index

In [ ]:
# Item-based recommendation function
def item_based_rec_assignment_3_iter3(picked_userid, number_of_similar_items, number_of_recommendations):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_item_norm_assignment_3_iter3[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title'].values.tolist()

  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_item_norm_assignment_3_iter3[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  

  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    # picked_movie_similarity_score = item_similarity_pearson[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    picked_movie_similarity_score = df_item_similarity_cosine_asign_3_iter3[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})

    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    # predicted_rating = round(np.average(picked_userid_watched_similarity['rating']),6)
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

In [ ]:
#recommendations for user 110

print("Recommended movies for user 110")
recommended_movies_user_110 = item_based_rec_assignment_3(picked_userid=110, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_110_df = pd.DataFrame(recommended_movies_user_110)
recommended_movies_user_110_df_list = recommended_movies_user_110_df[1].values.tolist()

Recommended movies for user 110


In [ ]:
print("Recommended movies for user 250")
recommended_movies_user_250 = item_based_rec_assignment_3(picked_userid=250, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_250_df = pd.DataFrame(recommended_movies_user_250)
recommended_movies_user_250_df_list = recommended_movies_user_250_df[1].values.tolist()

Recommended movies for user 250


In [ ]:
print("Recommended movies for user 401")
recommended_movies_user_401 = item_based_rec_assignment_3(picked_userid=401, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_401_df = pd.DataFrame(recommended_movies_user_401)
recommended_movies_user_401_df_list = recommended_movies_user_401_df[1].values.tolist()

Recommended movies for user 401


In [ ]:
output_list_group_3_iter3 = []
user_id_lists = [110,250,401]
for user in user_id_lists:
    recommended_movie = item_based_rec_assignment_3_iter3(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3_iter3.append(recommended_movie)

In [ ]:
#converting into dataframe
group_movies_df_iter3 =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df_iter3 = group_movies_df.rename(columns={0: 'movie', 1: 'score'})

In [ ]:
group_movies_df_iter3


,movie,score
0,Bridget Jones's Diary (2001),0.533963
1,Priceless (Hors de prix) (2006),0.522125
2,Speechless (1994),0.507170
3,Mallrats (1995),0.486892
4,Roommates (1995),0.457311
5,Princess Mononoke (Mononoke-hime) (1997),0.447869
6,"General, The (1926)",0.434774
7,Breakfast at Tiffany's (1961),0.422058
8,Lord of Illusions (1995),0.410941
9,Valentine's Day (2010),0.401128


because our alpha is low so we stick with average method for this iteration 3

In [ ]:
#using the average aggregation
df_mean = group_movies_df_iter3.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)
recommendations_for_3["score"] = recommendations_for_3["score"] * (1 - 0.0640479459038612)

In [ ]:
recommendations_for_3_list = recommendations_for_3['score'].values.tolist()
recommendations_for_3_list_top20 = recommendations_for_3_list[:20]

In [ ]:
#get satisfaction score of first iteration
satA1, satB1, satC1, alpha1, group_sat1= alpha_score(recommended_movies_user_110_df_list, recommended_movies_user_250_df_list, recommended_movies_user_401_df_list, recommendations_for_3_list_top20)

In [ ]:
satA1, satB1, satC1, alpha1, group_sat1

(2.8938416716817144,
 0.9615406931233097,
 2.5241365450679862,
 0.0640479459038612,
 2.1265063032910034)

# Iteration 4 + 5 = Similar to Iteration 1 + 2 + 3